In [2]:
from keras.applications.inception_v3 import InceptionV3
import os
from keras.layers import Flatten, Dense, AveragePooling2D
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

learning_rate = 0.0001
img_width = 299
img_height = 299
nbr_train_samples = 3019
nbr_validation_samples = 758
nbr_epochs = 30
batch_size = 48

train_data_dir = '/home/tyrael/Documents/ML/Fisheries/train_split'
val_data_dir = '/home/tyrael/Documents/ML/Fisheries/val_split'

FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

print('Loading InceptionV3 Weights ...')
InceptionV3_notop = InceptionV3(include_top=False, weights='imagenet',
                    input_tensor=None, input_shape=(299, 299, 3))

# Note that the preprocessing of InceptionV3 is:
# (x / 255 - 0.5) x 2

print('Adding Average Pooling Layer and Softmax Output Layer ...')
output = InceptionV3_notop.get_layer(index = -1).output  # Shape: (8, 8, 2048)
output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
output = Flatten(name='flatten')(output)
output = Dense(8, activation='softmax', name='predictions')(output)

InceptionV3_model = Model(InceptionV3_notop.input, output)
#InceptionV3_model.summary()

optimizer = SGD(lr = learning_rate, momentum = 0.9, decay = 0.0, nesterov = True)
InceptionV3_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

# autosave best Model
best_model_file = "./weights.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

# this is the augmentation configuration we will use for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True,
        classes = FishNames,
        class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        classes = FishNames,
        class_mode = 'categorical')

InceptionV3_model.fit_generator(
        train_generator,
        samples_per_epoch = nbr_train_samples,
        nb_epoch = nbr_epochs,
        validation_data = validation_generator,
        nb_val_samples = nbr_validation_samples,
        callbacks = [best_model])

Loading InceptionV3 Weights ...
Adding Average Pooling Layer and Softmax Output Layer ...
Found 3019 images belonging to 8 classes.
Found 758 images belonging to 8 classes.
Epoch 1/30
3019/3019 [==============================] - 154s - loss: 2.0395 - acc: 0.2269 - val_loss: 2.0495 - val_acc: 0.2045
Epoch 2/30
3019/3019 [==============================] - 89s - loss: 1.7899 - acc: 0.3935 - val_loss: 1.7457 - val_acc: 0.3865
Epoch 3/30
3019/3019 [==============================] - 90s - loss: 1.5874 - acc: 0.4978 - val_loss: 1.5280 - val_acc: 0.5317
Epoch 4/30
3019/3019 [==============================] - 90s - loss: 1.4280 - acc: 0.5548 - val_loss: 1.3561 - val_acc: 0.6003
Epoch 5/30
3019/3019 [==============================] - 90s - loss: 1.2708 - acc: 0.6022 - val_loss: 1.2366 - val_acc: 0.6425
Epoch 6/30
3019/3019 [==============================] - 90s - loss: 1.1590 - acc: 0.6472 - val_loss: 1.1222 - val_acc: 0.7071
Epoch 7/30
3019/3019 [==============================] - 91s - loss: 1.